In [1]:

import tensorflow as tf
import pandas as pd

from tqdm.auto import tqdm
import tensorflow as tf

In [2]:

!pip install transformers command

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for command: filename=Command-0.1.0-py3-none-any.whl size=6411 sha256=0d5a253d6bb15c0bcc69d87a2787b322655308450849bb8ce6ec146ca4287e61
  Stored in directory: c:\users\mx\appdata\local\pip\cache\wheels\a0\cb\b7\42f704db922b07a850570be67e5f92888d132e3b30a914d69e
Successfully built command

You should consider upgrading via the 'c:\users\mx\anaconda3\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4


In [ ]:

biden_df = pd.read_csv('cleaned_sample_100_biden.csv')
trump_df = pd.read_csv('cleaned_sample_100_trump.csv')
     
biden_df.loc[:,'who'] = 'biden'
trump_df.loc[:,'who'] = 'trump'

balance_data_df = pd.DataFrame(columns=biden_df.columns)
balance_data_df


biden_df = pd.read_csv('cleaned_sample_100_biden.csv')
trump_df = pd.read_csv('cleaned_sample_100_trump.csv')
     
biden_df.loc[:,'who'] = 'biden'
trump_df.loc[:,'who'] = 'trump'

balance_data_df = pd.DataFrame(columns=biden_df.columns)
balance_data_df
     
     
    
for state in states_list:
    trump_df_state = trump_df[trump_df['state_code']==state]
    biden_df_state = biden_df[biden_df['state_code']==state]
    
    n_sample = len(trump_df_state) if len(trump_df_state) < len(biden_df_state) else len(biden_df_state)
    
    if len(trump_df_state) < len(biden_df_state):
        balance_data_df = pd.concat([balance_data_df, trump_df_state])
        balance_data_df = pd.concat([balance_data_df, biden_df_state.sample(len(trump_df_state))])
    else:
        balance_data_df = pd.concat([balance_data_df, biden_df_state])
        balance_data_df = pd.concat([balance_data_df, trump_df_state.sample(len(biden_df_state))])

In [2]:

from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# This is for training model, don't have to run this

In [57]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84131840/84125825 [==============================] - 198s 2us/step


In [59]:
# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))

['labeledBow.feat', 'neg', 'pos', 'unsupBow.feat', 'urls_neg.txt', 'urls_pos.txt', 'urls_unsup.txt']


In [30]:
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [31]:

for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train.head()
     

,DATA_COLUMN,LABEL_COLUMN
0,Canadian director Vincenzo Natali took the art...,1
1,I gave this film 10 not because it is a superb...,1
2,I admit to being somewhat jaded about the movi...,1
3,"For a long time, 'The Menagerie' was my favori...",1
4,A truly frightening film. Feels as if it were ...,0


In [32]:

for j in test.take(1):
  test_feat = j[0].numpy()
  test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


In [63]:

InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)
     

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [33]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'
     

In [34]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(16).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(16)

C:\Users\Mx\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [40]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [41]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

#red_lr= ReduceLROnPlateau(monitor='loss',patience=1,verbose=1,factor=0.1, mode='min', min_lr=10**(-12))
mcp_save = ModelCheckpoint('weights1.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')

model.fit(train_data, epochs=2, verbose = 1, validation_data=validation_data, callbacks=[mcp_save])
     

Epoch 1/2
2500/2500 [==============================] - 1806s 711ms/step - loss: 0.0863 - accuracy: 0.9698 - val_loss: 0.4778 - val_accuracy: 0.8842
Epoch 2/2
2500/2500 [==============================] - 1805s 722ms/step - loss: 0.0326 - accuracy: 0.9896 - val_loss: 0.5079 - val_accuracy: 0.8788


In [68]:
model.save(f'bert_emo_finetuned')

INFO:tensorflow:Assets written to: bert_emo_finetuned\assets


INFO:tensorflow:Assets written to: bert_emo_finetuned\assets


# Using model

In [53]:
batch_size = 32
def emotion_classify(tweet_list):
    batch_idx = np.append(np.arange(0, len(tweet_list), batch_size), len(tweet_list))
    labels = np.array([])
    for i in tqdm(range(len(batch_idx)-1)):
        tf_batch = tokenizer(tweet_list[batch_idx[i]:batch_idx[i+1]], max_length=280, padding=True, truncation=True, return_tensors='tf')
        tf_outputs = model(tf_batch)
        tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
        labels = np.append(labels, tf.argmax(tf_predictions, axis=1))
        del tf_batch
    return labels

In [74]:
import os
# your directory here (might not be necessary)
# os.chdir("D:/socialweb/")

In [42]:
finetuned_bert = tf.keras.models.load_model('bert_emo_finetuned')
model.load_weights('weights1.hdf5')

In [45]:
# 0 is negative, 1 is positive
tf_batch = tokenizer(['He is a stupid man', 'He is a good man'], max_length=280, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf.nn.softmax(tf_outputs[0], axis=-1)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.98008007, 0.01991991],
       [0.00585578, 0.99414426]], dtype=float32)>

In [46]:
# example of not really working properly
tf_batch = tokenizer(['He is a bad man', 'He is a good man'], max_length=280, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf.nn.softmax(tf_outputs[0], axis=-1)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.01421283, 0.98578715],
       [0.00585578, 0.99414426]], dtype=float32)>

In [54]:
# Run the model on dataset
balance_data_df['sentiment'] = emotion_classify(list(balance_data_df['tweet']))

  0%|          | 0/2399 [00:00<?, ?it/s]

In [55]:
balance_data_df.to_csv('cleaned_sample_100_trumpbiden_sentiment.csv')  

In [58]:
pd.options.display.max_colwidth = 10000
balance_data_df

,tweet,long,lat,state_code,sentiment,who
3559,SWAtlasHoover This FOOL is at the Hoover! STOP DONATING MONEY TO THE HOOVER INSTITUTE. I wouldn't trust Atlas to treat your house pet never mind any human other than Trump! DO NOT DONATE TO THE HOOVER INSTITUTE!!!,-110.917506,43.357853,WY,0.0,biden
4220,"PamKirby RedPillLatina realDonaldTrump Pam, thank you for supporting our great president. He wants the best for all Americans. President Trump - doesn't pander to special interest groups - cannot be bought out - knows America is great amp wants to make it greater - knows lower taxes are better for the economy Trump",-107.568535,43.170026,WY,1.0,biden
4774,danicaamac CALFIRE CALFIRELNU SFGate KTVU CALFIRECHIEF GavinNewsom TeamPelosi GavinNewsom what is Calfire supporting Trump,-107.568535,43.170026,WY,0.0,biden
4781,SO Happy I requested my AbsenteeBallot bcuz nobody knew what the status of COVID19 would be come ElectionDay2020 but ALSO Voided my AbsenteeBallot and voted InPerson cuz who can trust Trump amp his efforts of VoterSuppression IVoted VoteInPerson MyBlueCountyIsntInYet,-106.280075,43.411391,WY,1.0,biden
4876,"Its Friday, and Grandpas doing his thinghelping all of us connect and Resist Lets join together in Resistance against Trump and the TrumpCrimeFamily VoteBlueToSaveAmerica and live in JoeBidensNeighborhood",-106.280075,43.411391,WY,1.0,biden
...,...,...,...,...,...,...
16974,All Biden BidenHarris BidenHarrisToSaveAmerica BidenHarris2020 Biden2020 Supporters must watch if you believe in freedom Full Documentary The American Republic vs the CCP Download our app to read more for free at,-106.650985,35.084103,NM,1.0,trump
11823,bennyjohnson JoeBiden Nothing to see here. - Facebook amp Twitter JoeBiden HunterBiden BidenCrimeFamily,-105.993007,34.570817,NM,0.0,trump
27003,"VP of Flint, MI City Council Endorses Trump ""Mr. Trump would be the best thing for poor folks""...""Obama said our water is fine...ain't nothing wrong with drinking lead!"" VIDEO Blexit JoeBiden",-105.993007,34.570817,NM,0.0,trump
20721,"A celebratory mimosa from us to you, joebiden and kamalaharris and to all of America! flexyourvote lgbtqrights hesfired presidentbiden vicepresidentharris",-106.650985,35.084103,NM,0.0,trump


In [59]:
df = pd.read_csv('cleaned_sample_100_trumpbiden_sentiment.csv')
df

,Unnamed: 0,tweet,long,lat,state_code,sentiment,who
0,3559,SWAtlasHoover This FOOL is at the Hoover! STOP DONATING MONEY TO THE HOOVER INSTITUTE. I wouldn't trust Atlas to treat your house pet never mind any human other than Trump! DO NOT DONATE TO THE HOOVER INSTITUTE!!!,-110.917507,43.357853,WY,0.0,biden
1,4220,"PamKirby RedPillLatina realDonaldTrump Pam, thank you for supporting our great president. He wants the best for all Americans. President Trump - doesn't pander to special interest groups - cannot be bought out - knows America is great amp wants to make it greater - knows lower taxes are better for the economy Trump",-107.568535,43.170026,WY,1.0,biden
2,4774,danicaamac CALFIRE CALFIRELNU SFGate KTVU CALFIRECHIEF GavinNewsom TeamPelosi GavinNewsom what is Calfire supporting Trump,-107.568535,43.170026,WY,0.0,biden
3,4781,SO Happy I requested my AbsenteeBallot bcuz nobody knew what the status of COVID19 would be come ElectionDay2020 but ALSO Voided my AbsenteeBallot and voted InPerson cuz who can trust Trump amp his efforts of VoterSuppression IVoted VoteInPerson MyBlueCountyIsntInYet,-106.280075,43.411391,WY,1.0,biden
4,4876,"Its Friday, and Grandpas doing his thinghelping all of us connect and Resist Lets join together in Resistance against Trump and the TrumpCrimeFamily VoteBlueToSaveAmerica and live in JoeBidensNeighborhood",-106.280075,43.411391,WY,1.0,biden
...,...,...,...,...,...,...,...
76755,16974,All Biden BidenHarris BidenHarrisToSaveAmerica BidenHarris2020 Biden2020 Supporters must watch if you believe in freedom Full Documentary The American Republic vs the CCP Download our app to read more for free at,-106.650985,35.084103,NM,1.0,trump
76756,11823,bennyjohnson JoeBiden Nothing to see here. - Facebook amp Twitter JoeBiden HunterBiden BidenCrimeFamily,-105.993007,34.570817,NM,0.0,trump
76757,27003,"VP of Flint, MI City Council Endorses Trump ""Mr. Trump would be the best thing for poor folks""...""Obama said our water is fine...ain't nothing wrong with drinking lead!"" VIDEO Blexit JoeBiden",-105.993007,34.570817,NM,0.0,trump
76758,20721,"A celebratory mimosa from us to you, joebiden and kamalaharris and to all of America! flexyourvote lgbtqrights hesfired presidentbiden vicepresidentharris",-106.650985,35.084103,NM,0.0,trump
